In [ ]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
random.seed(777777)

In [ ]:
from venues_and_time_schedules import (ROOMS, 
                                       LABS, 
                                       TIMES,
                                       WEEK_DAYS,
                                       merged_room_dict,
                                       room_dfs
                                      )
from course_details import (df_courses, 
                            course_codes,
                            course_lecturers, 
                            course_years,
                            course_groups,
                            course_rooms,
                            course_titles
                           )

from tt_utilities import exclude_clashing_indices

In [ ]:
[COM_titles, ACS_titles, STA_titles, MAT_titles, PHY_titles,CHE_titles, BOT_titles,
 ZOO_titles, PG_titles,COMMON_titles] = course_titles

In [ ]:
[UNIVERSAL_TITLES] = [COM_titles+ACS_titles+STA_titles+MAT_titles+PHY_titles+
                      CHE_titles+BOT_titles+ZOO_titles+PG_titles+COMMON_titles]
# UNIVERSAL_TITLES

In [ ]:
[COM_codes, ACS_codes, STA_codes, MAT_codes, PHY_codes,
 CHE_codes, BOT_codes, ZOO_codes, PG_codes, COMMON_codes] = course_codes

In [ ]:
[COM_years, ACS_years, STA_years, MAT_years, PHY_years,
 CHE_years, BOT_years, ZOO_years,PG_years, COMMON_years]= course_years
# COM_years

In [ ]:
[UNIVERSAL_YEARS] = [COM_years+ACS_years+STA_years+MAT_years+PHY_years+
                     CHE_years+BOT_years + ZOO_years+ PG_years+COMMON_years]
# UNIVERSAL_YEARS

In [ ]:
[COM_groups, ACS_groups, STA_groups, MAT_groups, PHY_groups,
 CHE_groups,BOT_groups,ZOO_groups,PG_groups, COMMON_groups]= course_groups
# COM_groups
# PG_groups

In [ ]:
[UNIVERSAL_GROUPS] = [COM_groups+ACS_groups+STA_groups+MAT_groups+PHY_groups+
                      CHE_groups+BOT_groups+ZOO_groups+PG_groups+COMMON_groups]
# UNIVERSAL_GROUPS

In [ ]:
mon_room_df, tue_room_df, wed_room_df, thu_room_df,fri_room_df = room_dfs
# fri_room_df
# mon_room_df

In [ ]:
[df_mat_courses,df_phy_courses,df_sta_courses,df_acs_courses,df_com_courses,
 df_che_courses,df_bot_courses,df_zoo_courses,df_pg_courses, df_common_courses] = df_courses
# df_pg_courses

In [ ]:
# df_che_courses

In [ ]:
# df_che_courses["CODE"].loc[0]

In [ ]:
[COM_codes, ACS_codes, STA_codes, MAT_codes, PHY_codes, 
 CHE_codes,BOT_codes,ZOO_codes, PG_codes,COMMON_codes]  = course_codes
# PHY_list

In [ ]:
[COM_list, ACS_list, STA_list, MAT_lists, PHY_list, 
 CHE_list,BOT_list,ZOO_list, PG_list,COMMON_list] = [list(codes) for codes in course_codes]
# PG_list

In [ ]:
[COM_lects, ACS_lects, STA_lects, MAT_lects, PHY_lects,
 CHE_lects,BOT_lects,ZOO_lects, PG_lects, COMMON_lects] = course_lecturers
# PG_lects

In [ ]:
[UNIVERSAL] = [list(COM_codes)+list(ACS_codes)+list(STA_codes)+list(MAT_codes)+list(PHY_codes)
               +list(CHE_codes)+list(BOT_codes)+list(ZOO_codes)+list(PG_codes)+list(COMMON_codes)]
# UNIVERSAL

In [ ]:
[UNIVERSAL_LECTS] = [list(COM_lects)+list(ACS_lects)+list(STA_lects)+list(MAT_lects)+list(PHY_lects)+
                     list(CHE_lects)+list(BOT_lects)+list(ZOO_lects)+list(PG_lects)+list(COMMON_lects)]
# UNIVERSAL_LECTS

In [ ]:
[COM_groups, ACS_groups, STA_groups,MAT_groups, PHY_groups, 
 CHE_groups, BOT_groups,ZOO_groups,PG_groups,COMMON_groups] = course_groups

In [ ]:
# COM_groups

In [ ]:
[UNIVERSAL_GROUPS] = [COM_groups+ ACS_groups+ STA_groups + MAT_groups+ PHY_groups + 
                      CHE_groups+ BOT_groups+ZOO_groups+PG_groups+COMMON_groups]
# UNIVERSAL_GROUPS 

In [ ]:
[COM_rooms, ACS_rooms, STA_rooms,MAT_rooms, PHY_rooms,
 CHE_rooms, BOT_rooms,ZOO_rooms,PG_rooms,COMMON_rooms]=course_rooms

In [ ]:
[UNIVERSAL_ROOMS] = [COM_rooms+ACS_rooms+STA_rooms+MAT_rooms+PHY_rooms+
                     CHE_rooms+BOT_rooms+ZOO_rooms+PG_rooms+COMMON_rooms]
# UNIVERSAL_ROOMS

In [ ]:
scheduled, scheduled_2, unscheduled, unscheduled_2  = [], [], [], []

In [ ]:
LAB_codes = COM_list + PHY_list+ CHE_list+ BOT_list+ ZOO_list
# LAB_codes

In [ ]:
def schedule_request(room_dfs, 
                     day_1='Monday',
                     day_2='Tuesday', 
                     time_1='8.00AM-9.00AM', 
                     time_2='10.00AM-11.00AM', 
                     code='CSC 122', 
                     room='SL1'):
    
    global scheduled, scheduled_2, unscheduled, unscheduled_2 
    TIMES_array = np.array(TIMES)
    
#     time1_idx = TIMES.index(time_1)
#     time2_idx = TIMES.index(time_2)
    
    day1_idx = WEEK_DAYS.index(day_1)
    day2_idx = WEEK_DAYS.index(day_2)
       
    df_rooms_1 = room_dfs[day1_idx]
    df_rooms_2 = room_dfs[day2_idx]
    
    ## Fetch code index in folder 'UNIVERSAL'   
    idx = UNIVERSAL.index(code)
    
    ## Fetch rooms allocated to code in folder 'UNIVERSAL_ROOMS'
    code_rooms = UNIVERSAL_ROOMS[idx]
#     print(code_rooms)
    
    code_idxes =[]
    for other_code in UNIVERSAL:   
        if other_code==code:
            code_idxes.append(UNIVERSAL.index(other_code))

    ## Fetch lecturer allocated to code in folder 'UNIVERSAL_GROUPS'
    lec = UNIVERSAL_LECTS[idx]
    
    ## Fetch code year in folder 'UNIVERSAL_YEARS'
    code_year= UNIVERSAL_YEARS[idx]
    
    ## Fetch groups associated with code
    code_groups = UNIVERSAL_GROUPS[idx]
#     print(code_groups )
    
    ## Time slots - normal hours and lab practical hours.
    shedule1_time_indices = np.arange(2,12,2) 
#     print('shedule1_time_indices:',shedule1_time_indices)
    time1_idx = np.where(TIMES_array==time_1)[0][0]+1
    time2_idx = np.where(TIMES_array==time_2)[0][0]+1
#     print('time1_idx:', time1_idx, 'time2_idx:', time2_idx)
    
    if code in LAB_codes:
        shedule2_time_indices = np.arange(1,12,3)
        allocated_1= list(df_rooms_1.iloc[time1_idx].values)+list(df_rooms_1.iloc[time1_idx+1].values)+list(df_rooms_1.iloc[time1_idx+2].values)
        allocated_2= list(df_rooms_2.iloc[time2_idx].values)+list(df_rooms_2.iloc[time2_idx+1].values)+list(df_rooms_2.iloc[time2_idx+2].values)
    else:
        shedule2_time_indices = np.arange(2,12,2) 
        allocated_1= list(df_rooms_1.iloc[time1_idx].values)+list(df_rooms_1.iloc[time1_idx+1].values)
        allocated_2= list(df_rooms_2.iloc[time2_idx].values)+list(df_rooms_2.iloc[time2_idx+1].values)
    
#     print(ROOMS)
#     print(LABS)
    VENUES = ROOMS + LABS
#     print(VENUES)
    room_idx = random.choice([VENUES.index(code_room) for code_room in code_rooms])
#     print(room_idx)  
    time_slots = [time1_idx,time1_idx+1]
    if code in LAB_codes:
        time_slots_2 = [time2_idx,time2_idx+1,time2_idx+2]
        if code in COM_list:
            lab_idx=VENUES.index('COM-LAB')
        elif code in PHY_list:
            lab_idx=VENUES.index('PHY-LAB (SL2)')
        elif code in CHE_list:
            lab_idx=VENUES.index('CHE-LAB')
        elif code in BOT_list:
            lab_idx=VENUES.index('BOT-LAB')
        elif code in ZOO_list:
            lab_idx=VENUES.index('ZOO-LAB')     
    else:
        time_slots_2 = [time2_idx,time2_idx+1]        
        
    if code not in scheduled:
        exclude_idx = []
        for item in allocated_1:
            if item  in UNIVERSAL:
                item_idx = UNIVERSAL.index(item)
                if UNIVERSAL_LECTS[item_idx]==lec:
                    exclude_idx.append(item_idx)
                if UNIVERSAL_YEARS[item_idx]==code_year:
                    for groups in UNIVERSAL_GROUPS[item_idx]:
                        for group in groups:
                            for code_group in code_groups:
                                if group== code_group:
                                    exclude_idx.append(item_idx)

        if len(exclude_idx)==0:
            if df_rooms_1.iloc[[time1_idx],[room_idx]].values[0][0] not in UNIVERSAL:
                    df_rooms_1.iloc[time_slots,[room_idx]]= code
                    scheduled.append(code)                                               
            
        else:
            time1_idx_available = []
            for i_ in shedule1_time_indices:
                if df_rooms_1.iloc[[i_],[room_idx]].values[0][0] not in UNIVERSAL:
                    time1_idx_available.append(i_)
                else:
                    for code_room in code_rooms:
                        if code_room != ROOMS[room_idx]:
                            i_= ROOMS.index(code_room)
                            time1_idx_available.append(i_)
                        
                                                       
            if len(time1_idx_available)!=0:
                i = random.choice(time1_idx_available)
                df_rooms_1.iloc[[i,i+1],[room_idx]]= code
                scheduled.append(code)  
            else:
                unscheduled.append(code)

            
    if code not in scheduled_2:
        exclude_idx = []
        if code in LAB_codes:
            venue_idx=lab_idx
        else:
            venue_idx=room_idx
            
        for item in allocated_2:
            if item  in UNIVERSAL:
                item_idx = UNIVERSAL.index(item)
                if UNIVERSAL_LECTS[item_idx]==lec:
                    exclude_idx.append(item_idx)
                if UNIVERSAL_YEARS[item_idx]==code_year:
                    for groups in UNIVERSAL_GROUPS[item_idx]:
                        for group in groups:
                            for code_group in code_groups:
                                if group == code_group:
                                    exclude_idx.append(item_idx)
                                    
        if len(exclude_idx)==0:
            if df_rooms_2.iloc[[time2_idx],[venue_idx]].values[0][0] not in UNIVERSAL:
                    df_rooms_2.iloc[time_slots_2,[venue_idx]]= code
                    scheduled.append(code)                                               
            
        else:
            time2_idx_available = []
            for i_ in shedule2_time_indices:
                if df_rooms_2.iloc[[i_],[venue_idx]].values[0][0] not in UNIVERSAL:
                    time2_idx_available.append(i_)
                                                       
            if len(time2_idx_available)!=0:
                i = random.choice(time2_idx_available)
                df_rooms_2.iloc[[i,i+1],[venue_idx]]= code
                scheduled_2.append(code)  
            else:
                unscheduled_2.append(code)
                
    return df_rooms_1, df_rooms_2

In [ ]:
## Dr. Mutai Charles
schedule_request(room_dfs,
                 day_1='Monday',
                 day_2='Thursday',
                 time_1='8.00AM-9.00AM',
                 time_2='10.00AM-11.00AM',
                 code='STA 124',
                 room='LR3')

schedule_request(room_dfs,
                 day_1='Monday',
                 day_2='Thursday',
                 time_1='10.00AM-11.00AM',
                 time_2='8.00AM-9.00AM',
                 code='STA 122',
                 room='LR3')

schedule_request(room_dfs,
                 day_1='Monday',
                 day_2='Thursday',
                 time_1='12.00PM-1.00PM',
                 time_2='12.00PM-1.00PM',
                 code= 'STA 425',
                 room='LR3')

## Dr. Jacq Mak: SZL 121, ZOO 213, ZOO 315, ZOO 421E
schedule_request(room_dfs, 
                 day_1='Monday',
                 day_2='Thursday', 
                 time_1='8.00AM-9.00AM', 
                 time_2='7.00AM-8.00AM',
                 code='SZL 121')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Thursday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='10.00AM-11.00AM',
                 code='ZOO 213')

schedule_request(room_dfs, 
                 day_1='Thursday', 
                 day_2='Monday', 
                 time_1='2.00PM-3.00PM',
                 time_2='1.00PM-2.00PM',
                 code='ZOO 421E')

schedule_request(room_dfs, 
                 day_1='Thursday', 
                 day_2='Monday', 
                 time_1='4.00PM-5.00PM', 
                 time_2='4.00PM-5.00PM',
                 code='ZOO 315')

## Dr. Bill
schedule_request(room_dfs,
                 day_1='Tuesday', 
                 time_1='10.00AM-11.00AM', 
                 day_2='Wednesday', 
                 time_2='4.00PM-5.00PM', 
                 code='MIC 212', 
                 room='LR3')

schedule_request (room_dfs, 
                  day_1='Tuesday', 
                  time_1='8.00AM-9.00AM',
                  day_2='Wednesday', 
                  time_2='7.00AM-8.00AM',
                  code='BOT 320', 
                  room='LR3')

schedule_request (room_dfs, 
                  day_1='Tuesday', 
                  time_1='4.00PM-5.00PM',
                  day_2='Wednesday', 
                  time_2='10.00AM-11.00AM', 
                  code='MIC 211', 
                  room='LR3')

schedule_request (room_dfs, 
                  day_1='Tuesday',
                  time_1='12.00PM-1.00PM', 
                  day_2='Wednesday', 
                  time_2='1.00PM-2.00PM', 
                  code='MIC 418',
                  room='LR3')
#Dr. EM 
schedule_request(room_dfs,
                 day_1='Monday', 
                 day_2='Tuesday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='2.00PM-3.00PM', 
                 code='COM 223', 
                  room='AH1')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Tuesday', 
                 time_1='12.00PM-1.00PM',
                 time_2='10.00AM-11.00AM',
                 code='COM 323E', 
                  room='AH1')
     
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Tuesday', 
                 time_1='2.00PM-3.00PM', 
                 time_2='12.00PM-1.00PM',  
                 code='COM 424E', 
                  room='AH1')

#Dr.L
schedule_request(room_dfs, 
                 day_1='Wednesday', 
                 day_2='Tuesday', 
                 time_1='11.00AM-12.00PM', 
                 time_2='1.00PM-2.00PM',  
                 code='MIC 215')

schedule_request(room_dfs, 
                 day_1='Wednesday', 
                 day_2='Thursday', 
                 time_1='1.00PM-2.00PM', 
                 time_2='11.00AM-12.00PM',  
                 code='MIC 414'
                )

schedule_request(room_dfs, 
                 day_1='Wednesday', 
                 day_2='Thursday', 
                 time_1='9.00AM-10.00AM', 
                 time_2='8.00AM-9.00AM',  
                 code='ZOO 415' 
                 )

schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Wednesday', 
                 time_1='3.00PM-4.00PM', 
                 time_2='7.00AM-8.00AM',  
                 code='ZOO 417')

#### Dr. Z
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Tuesday', 
                 time_1='2.00PM-3.00PM', 
                 time_2='7.00AM-8.00AM',  
                 code='CHE 403')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Tuesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='10.00AM-11.00AM',  
                 code='BIO 416')

schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Monday', 
                 time_1='2.00PM-3.00PM', 
                 time_2='4.00PM-5.00PM',  
                 code='CHE 416')

schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Monday', 
                 time_1='4.00PM-5.00PM', 
                 time_2='7.00AM-8.00AM',  
                 code='CHE 423')

#### Prof. R
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Friday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='7.00AM-8.00AM',  
                 code='COM 215')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Friday', 
                 time_1='8.00AM-9.00AM', 
                 time_2='10.00AM-11.00AM',  
                 code='PHY 430E/COM 422')

#### Dr. K
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Thursday', 
                 time_1='8.00AM-9.00AM', 
                 time_2='10.00AM-11.00AM',  
                 code='MAT 214')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Thursday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='2.00PM-3.00PM',  
                 code='MAT 425')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Thursday', 
                 time_1='2.00PM-3.00PM', 
                 time_2='8.00AM-9.00AM',  
                 code='MAT 419')

### Dr. Ch.
schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Thursday', 
                 time_1='2.00PM-3.00PM', 
                 time_2='12.00PM-1.00PM',  
                 code='ACS 407')

schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Thursday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='2.00PM-3.00PM',  
                 code='ACS 305')

schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Thursday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='10.00AM-11.00AM',  
                 code='STA 216')

### Dr.R
schedule_request(room_dfs, 
                 day_1='Tuesday', 
                 day_2='Wednesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='10.00AM-11.00AM',  
                 code='MIC 415')

### Dr. Silver R.
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Wednesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='10.00AM-11.00AM',  
                 code='STA 427')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Wednesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='12.00PM-1.00PM',  
                 code='STA 415')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Wednesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='8.00AM-9.00AM',  
                 code='STA 217')

### Dr. K.
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Wednesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='4.00PM-5.00PM',  
                 code='STA 423')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Wednesday', 
                 time_1='4.00PM-5.00PM', 
                 time_2='10.00AM-11.00AM',  
                 code='STA 224')

schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Wednesday', 
                 time_1='2.00PM-3.00PM', 
                 time_2='8.00AM-9.00AM',  
                 code='STA 937')

### Dr. Rh
schedule_request(room_dfs, 
                 day_1='Monday', 
                 day_2='Thursday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='8.00AM-9.00AM',  
                 code='SMA 121')

## Dr. Rs 
schedule_request(room_dfs, 
                 day_1='Wednesday', 
                 day_2='Friday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='7.00AM-8.00AM',  
                 code='MIC 313')

schedule_request(room_dfs, 
                 day_1='Wednesday', 
                 day_2='Friday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='4.00PM-5.00PM',  
                 code='MIC 419')

schedule_request(room_dfs, 
                 day_1='Friday', 
                 day_2='Wednesday', 
                 time_1='10.00AM-11.00AM', 
                 time_2='7.00AM-8.00AM',  
                 code='BOT 211')

schedule_request(room_dfs, 
                 day_1='Friday', 
                 day_2='Wednesday', 
                 time_1='12.00PM-1.00PM', 
                 time_2='4.00PM-5.00PM',  
                 code='BOT 317')

In [ ]:
room_dfs[0]

In [ ]:
def schedule_course(room_dfs, code='CSC 122'):
    
    global scheduled, unscheduled
    
    TIMES_array = np.array(TIMES)

    
    ## Fetch code index in folder 'UNIVERSAL'   
    idx = UNIVERSAL.index(code)
    
    ## Fetch rooms allocated to code in folder 'UNIVERSAL_ROOMS'
    code_rooms = UNIVERSAL_ROOMS[idx]
#     print(code_rooms)
    
    code_idxes =[]
    for other_code in UNIVERSAL:   
        if other_code==code:
            code_idxes.append(UNIVERSAL.index(other_code))

    ## Fetch lecturer allocated to code in folder 'UNIVERSAL_GROUPS'
    lec = UNIVERSAL_LECTS[idx]
    
    ## Fetch code year in folder 'UNIVERSAL_YEARS'
    code_year= UNIVERSAL_YEARS[idx]
    
    ## Fetch groups associated with code
    code_groups = UNIVERSAL_GROUPS[idx]
#     print(code_groups )
    
    ## Time slots - normal hours and lab practical hours.
    shedule1_time_indices = np.arange(2,12,2) 
    
#     print('shedule1_time_indices:',shedule1_time_indices)
    time1_idx = random.choice(shedule1_time_indices)+1
#     print('time1_idx:', time1_idx)

    allocated_1= list(room_dfs.iloc[time1_idx].values)+list(room_dfs.iloc[time1_idx+1].values)

    room_idx = random.choice([ROOMS.index(code_room) for code_room in code_rooms])


    time_slots = [time1_idx,time1_idx+1]       
        
    if code not in scheduled:
        exclude_idx = []
        for item in allocated_1:
            if item  in UNIVERSAL:
                item_idx = UNIVERSAL.index(item)
                if UNIVERSAL_LECTS[item_idx]==lec:
                    exclude_idx.append(item_idx)
                if UNIVERSAL_YEARS[item_idx]==code_year:
                    for groups in UNIVERSAL_GROUPS[item_idx]:
                        for group in groups:
                            for code_group in code_groups:
                                if group== code_group:
                                    exclude_idx.append(item_idx)

        if len(exclude_idx)==0:
            if room_dfs.iloc[[time1_idx],[room_idx]].values[0][0] not in UNIVERSAL:
                    room_dfs.iloc[time_slots,[room_idx]]= code
                    scheduled.append(code)                                               
            
        else:
            time1_idx_available = []
            for i_ in shedule1_time_indices:
                if room_dfs.iloc[[i_],[room_idx]].values[0][0] not in UNIVERSAL:
                    time1_idx_available.append(i_)
                else:
                    for code_room in code_rooms:
                        if code_room != ROOMS[room_idx]:
                            i_= ROOMS.index(code_room)
                            time1_idx_available.append(i_)
                        
                                                       
            if len(time1_idx_available)!=0:
                i = random.choice(time1_idx_available)
                room_dfs.iloc[[i,i+1],[room_idx]]= code
                scheduled.append(code)  
            else:
                unscheduled.append(code)
                
    return room_dfs

In [ ]:
print(unscheduled)
print(unscheduled_2)

In [ ]:
for course in UNIVERSAL:
#     print(course)
    schedule_course(random.choice(room_dfs), course)

In [ ]:
def second_schedule(room_dfs, code='CSC 122'):
    
    global  scheduled_2, unscheduled_2 
    TIMES_array = np.array(TIMES)
    
    ## Fetch code index in folder 'UNIVERSAL'   
    idx = UNIVERSAL.index(code)
    
    ## Fetch rooms allocated to code in folder 'UNIVERSAL_ROOMS'
    code_rooms = UNIVERSAL_ROOMS[idx]
#     print(code_rooms)
    
    code_idxes =[]
    for other_code in UNIVERSAL:   
        if other_code==code:
            code_idxes.append(UNIVERSAL.index(other_code))

    ## Fetch lecturer allocated to code in folder 'UNIVERSAL_GROUPS'
    lec = UNIVERSAL_LECTS[idx]
    
    ## Fetch code year in folder 'UNIVERSAL_YEARS'
    code_year= UNIVERSAL_YEARS[idx]
    
    ## Fetch groups associated with code
    code_groups = UNIVERSAL_GROUPS[idx]
#     print(code_groups )

    
    if code in LAB_codes:
        shedule2_time_indices = np.arange(1,12,3)
        time2_idx = random.choice(shedule2_time_indices)
        allocated_2= list(room_dfs.iloc[time2_idx].values)+list(room_dfs.iloc[time2_idx+1].values)+list(room_dfs.iloc[time2_idx+2].values)
    else:
        shedule2_time_indices = np.arange(2,12,2) 
        time2_idx = random.choice(shedule2_time_indices)+1
        allocated_2= list(room_dfs.iloc[time2_idx].values)+list(room_dfs.iloc[time2_idx+1].values)        

    VENUES = ROOMS + LABS

    room_idx = random.choice([VENUES.index(code_room) for code_room in code_rooms])

    if code in LAB_codes:
        time_slots_2 = [time2_idx,time2_idx+1,time2_idx+2]
        if code in COM_list:
            lab_idx=VENUES.index('COM-LAB')
        elif code in PHY_list:
            lab_idx=VENUES.index('PHY-LAB (SL2)')
        elif code in CHE_list:
            lab_idx=VENUES.index('CHE-LAB')
        elif code in BOT_list:
            lab_idx=VENUES.index('BOT-LAB')
        elif code in ZOO_list:
            lab_idx=VENUES.index('ZOO-LAB')     
    else:
        time_slots_2 = [time2_idx,time2_idx+1]        
        
 
            
    if code not in scheduled_2:
        exclude_idx = []
        if code in LAB_codes:
            venue_idx=lab_idx
        else:
            venue_idx=room_idx
            
        for item in allocated_2:
            if item  in UNIVERSAL:
                item_idx = UNIVERSAL.index(item)
                if UNIVERSAL_LECTS[item_idx]==lec:
                    exclude_idx.append(item_idx)
                if UNIVERSAL_YEARS[item_idx]==code_year:
                    for groups in UNIVERSAL_GROUPS[item_idx]:
                        for group in groups:
                            for code_group in code_groups:
                                if group == code_group:
                                    exclude_idx.append(item_idx)
                                    
        if len(exclude_idx)==0:
            if room_dfs.iloc[[time2_idx],[venue_idx]].values[0][0] not in UNIVERSAL:
                    room_dfs.iloc[time_slots_2,[venue_idx]]= code
                    scheduled.append(code)                                               
            
        else:
            time2_idx_available = []
            for i_ in shedule2_time_indices:
                if room_dfs.iloc[[i_],[venue_idx]].values[0][0] not in UNIVERSAL:
                    time2_idx_available.append(i_)
                                                       
            if len(time2_idx_available)!=0:
                i = random.choice(time2_idx_available)
                room_dfs.iloc[[i,i+1],[venue_idx]]= code
                scheduled_2.append(code)  
            else:
                unscheduled_2.append(code)
                
    return room_dfs

In [ ]:
second_schedule(fri_room_df, code='CSC 122')

In [ ]:
for course in UNIVERSAL:
    second_schedule(random.choice(room_dfs), course)

In [ ]:
# scheduled_2

In [ ]:
mon_room_df

In [ ]:
tue_room_df

In [ ]:
wed_room_df

In [ ]:
thu_room_df

In [ ]:
fri_room_df

In [ ]:
mon_room_df.loc[:, ROOMS].T

In [ ]:
mon_room_df.T

In [ ]:
mon_room_df.loc[:, LABS]

In [ ]:
room_frames = [ mon_room_df.loc[:, ROOMS].T,  tue_room_df.loc[:, ROOMS].T,  
               wed_room_df.loc[:, ROOMS].T, thu_room_df.loc[:, ROOMS].T,  fri_room_df.loc[:, ROOMS].T]
lab_frames = [ mon_room_df.loc[:, LABS].T,  tue_room_df.loc[:, LABS].T,  
               wed_room_df.loc[:, LABS].T, thu_room_df.loc[:, LABS].T,  fri_room_df.loc[:, LABS].T]
venue_frames = [ mon_room_df.T,  tue_room_df.T, wed_room_df.T, thu_room_df.T,  fri_room_df.T]

In [ ]:
room_frames[0]

In [ ]:
lab_frames[0]

In [ ]:
venue_frames[0]

In [ ]:
all_week_courses=[]
for i in range(len(WEEK_DAYS)):
    day_courses=[]
    for room in ROOMS:
        for course in list(room_frames[i].loc[room].values):
            if course not in day_courses:
                day_courses.append(course)
    all_week_courses.append(day_courses)
        
            
weekly_df_lecs = []
for day_courses in all_week_courses:
    lecs = []
    for code in day_courses:
        if code in UNIVERSAL:
            lecs.append([code,UNIVERSAL_TITLES[UNIVERSAL.index(code)],
                         UNIVERSAL_LECTS[UNIVERSAL.index(code)], UNIVERSAL_GROUPS[UNIVERSAL.index(code)]])
    weekly_df_lecs.append(pd.DataFrame(lecs, columns=['CODE','TITLE','LECTURER', 'GROUPS']))
weekly_df_lecs[0]

In [ ]:
# all_week_courses

In [ ]:
with pd.ExcelWriter('/Users/NRonoh/Documents/SSAS_TT/SSAS_TT_Jan2024.xlsx') as writer:
    for i in range(len(WEEK_DAYS)):
        venue_frames[i].to_excel(writer, sheet_name=WEEK_DAYS[i])
        weekly_df_lecs[i].to_excel(writer, sheet_name=WEEK_DAYS[i],
                                   startrow=len(venue_frames[i]) + 2)
#         room_frames[i].to_excel(writer, sheet_name=WEEK_DAYS[i])
#         lab_frames[i].to_excel(writer, sheet_name=WEEK_DAYS[i], startrow=len(room_frames[i]) + 2)
#         weekly_df_lecs[i].to_excel(writer, sheet_name=WEEK_DAYS[i],
#                                    startrow=len(room_frames[i]) + len(lab_frames[i])+ 4)

In [ ]:
# function
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()


In [ ]:
details_sheets = ['Mon_details','Tue_details','Wed_details','Thu_details','Fri_details']  
dfs_tabs(weekly_df_lecs, details_sheets, 'details.xlsx')

In [ ]:
TT_sheets = ['Mon_TT','Tue_TT','Wed_TT','Thu_TT','Fri_TT']  
dfs_tabs(venue_frames, TT_sheets, 'TT_draft2_19thJan2024.xlsx')